In [ ]:
import pandas as pd

df = pd.read_csv(
    "/Users/danylewin/thingies/university/CS Workshop/Finney/data/PassFInder_Password_Dataset/password_test.csv",
    header=None,
    names=["text", "label"],
)
df.sample(20)

In [ ]:
def display(data):
    # Exclude 'text' and 'label' columns
    cols_to_sum = data.columns.difference(["text", "label"])

    # Group by label and sum the rest
    result = data.groupby("label")[cols_to_sum].sum()

    print(result)

In [ ]:
df["special"] = df["text"].str.contains("[!@#\$\^\&\*\(\)_\+\[\]'\"\;\/\,\>\<\\\|\{\}\?\.]", regex=True)
df["special_end"] = df["text"].str.contains("[^a-zA-Z0-9]$", regex=True)
df["upper_start"] = df["text"].str.contains("^[A-Z]", regex=True)
df["space"] = df["text"].str.contains(" ")
df.sample(10)
# df.groupby("label")[["special", "special_end", "upper_start", "space"]].sum()
display(df)

In [ ]:
df["all_hexa"] = df["text"].str.contains("^[0-9A-Fa-f]*[A-Fa-f][0-9A-Fa-f]*$", regex=True)
df["byte"] = df["text"].str.contains(r"\\{1,2}\w{3,4}(?!\w)", regex=True)
# df["new_line"] = df["text"].str.contains(r"\n",regex=True)
display(df)

In [ ]:
df[df.fillna(0).all_hexa > 0]

In [ ]:
df["path_sub"] = df["text"].str.contains(r"(^/\./)|(^\./)", regex=True)
df["path_relative"] = df["text"].str.contains(r"(^/\.\./)|(^\.\./)", regex=True)
display(df)


In [ ]:
df["letter_number_symbol"] = df["text"].str.contains(r"^[A-Za-z]+[0-9]+\W?$", regex=True)
df["letter_symbol_number"] = df["text"].str.contains(r"^[A-Za-z]+\W?[0-9]+$", regex=True)
display(df)

In [ ]:
df["year"] = df["text"].str.contains(r"\b(19[0-9]{2}|20[0-9]{2}|2100)\b", regex=True)
df["date"] = df["text"].str.contains(r"\d{4}-\d{2}-\d{2}", regex=True)
display(df)

In [ ]:
df["xml"] = df["text"].str.contains(r"<.{1,3}>", regex=True)
df["period"] = df["text"].str.contains("\.")
df["double_colon"] = df["text"].str.contains("::")
df["question"] = df["text"].str.contains("\?")
df["percent"] = df["text"].str.contains("%")
df["arrow"] = df["text"].str.contains("->")
df["dunder"] = df["text"].str.contains("__")
df["double_equal"] = df["text"].str.contains("==")
df["triple_equal"] = df["text"].str.contains("===")
df["double_slash"] = df["text"].str.contains("//")
df["backslash"] = df["text"].str.contains("\\\\")
df["double_backslash"] = df["text"].str.contains("\\\\\\\\")
df["newline"] = df["text"].str.contains(r"\\n", regex=True)
display(df)

In [ ]:
df["balanced_parentheses"] = (
        (df["text"].str.count(r"\(") == df["text"].str.count(r"\)")) &
        (df["text"].str.count(r"\(") > 0)
)
df["balanced_parentheses_square"] = (
        (df["text"].str.count(r"\[") == df["text"].str.count(r"\]")) &
        (df["text"].str.count(r"\[") > 0)
)
df["balanced_parentheses_curl"] = (
        (df["text"].str.count(r"\{") == df["text"].str.count(r"\}")) &
        (df["text"].str.count(r"\{") > 0)
)
display(df)

In [ ]:
df.sample(10)

In [ ]:
import re

In [ ]:
df["longest_upper"] = df["text"].apply(
    lambda x: max((len(m) for m in re.findall(r"[A-Z]+", str(x))), default=0)
)
display(df)

In [ ]:
df["longest_vowels"] = df["text"].apply(
    lambda x: max((len(m) for m in re.findall(r"[aeiouAEIOU]+", str(x))), default=0)
)
df["longest_cons"] = df["text"].apply(
    lambda x: max((len(m) for m in re.findall(r"[bcdfghjklmnpqrstvxzBCDFGHJKLMNPQRSTVXZ]+", str(x))), default=0)
)
df["longest_hexa"] = df["text"].apply(
    lambda x: max((len(m) for m in re.findall(r"[0-9A-Fa-f]*[A-Fa-f][0-9A-Fa-f]*", str(x))), default=0)
)
display(df)

In [ ]:
df["digit_fraction"] = df["text"].apply(
    lambda x: sum(c.isdigit() for c in str(x)) / len(str(x))
)
df["vowel_fraction"] = df["text"].apply(
    lambda x: sum(c in set("aeiouAEIOU") for c in str(x)) / len(str(x))
)
df["nonword_fraction"] = df["text"].apply(
    lambda x: len(re.findall(r"\W", str(x))) / len(str(x))
)
display(df)

In [ ]:
df["word_length_mod_4"] = df["text"].apply(
    lambda x: sum(1 for m in re.findall(r"\w+", str(x)) if len(m) % 4 == 0)
)
display(df)

In [ ]:
import numpy as np

p1 = r'^[A-Za-z]+(?:_[A-Za-z_]+)+$'  # snake
p2 = r'^[A-Z][a-z]+(?:[A-Z][a-z]+)+$'  # UpperCamel
p3 = r'^[a-z]+(?:[A-Z][a-z]+)+$'  # lowerCamel
p4 = r'^[A-Za-z]+(?:-[A-Za-z-]+)+$'  # kebab-like (at least one "-")
p5 = r'^[A-Z]+$'  # ALLCAPS

conds = [
    df["text"].str.fullmatch(p1, na=False),
    df["text"].str.fullmatch(p2, na=False),
    df["text"].str.fullmatch(p3, na=False),
    df["text"].str.fullmatch(p4, na=False),
    df["text"].str.fullmatch(p5, na=False),
]

df["format"] = np.select(conds, [1, 2, 3, 4, 5], default=0).astype(int)
display(df)

In [ ]:
# words = pd.read_csv("/Users/danylewin/thingies/university/CS Workshop/Finney/data/words.txt", header=None, names=["word"])
# wordset = set(words["word"].astype(str))

english_words = set()
with open("data/words.txt", "r") as f:
    for line in f.readlines():
        english_words.add(line.strip().casefold())
temp = "|".join(map(re.escape, english_words))
english_pattern = re.compile(rf"\b(?:{temp})\b")

df["word"] = df["text"].astype(str).str.contains(english_pattern, na=False, regex=True)
display(df)

In [ ]:
df["word_count"] = df["text"].astype(str).str.findall(english_pattern).str.len()
display(df)

In [ ]:
keywords = set()
with open("data/keywords.txt", "r") as f:  # taken from https://github.com/e3b0c442/keywords?tab=readme-ov-file
    for line in f.readlines():  # and from https://www.ibm.com/docs/en/i/7.6.0?topic=extensions-standard-c-library-functions-table-by-name
        keywords.add(line.strip().casefold())
temp = "|".join(map(re.escape, keywords))
keyword_pattern = re.compile(rf"\b(?:{temp})\b")

df["keyword"] = df["text"].astype(str).str.contains(keyword_pattern, na=False, regex=True)
df["keyword_count"] = df["text"].astype(str).str.findall(keyword_pattern).str.len()
display(df)

In [ ]:
domains = set()  # taken from https://github.com/datasets/top-level-domain-names/blob/main/data/top-level-domain-names.csv?plain=1
with open("data/domains.txt", "r") as f:
    for line in f.readlines():
        domains.add(line.strip().casefold())
temp = "|".join(map(re.escape, domains))
url_pattern = re.compile(rf"(?:{temp})\b")

df["likely_url"] = df["text"].astype(str).str.contains(url_pattern, na=False)
display(df)

In [ ]:
extensions = set()  # taken from https://gist.github.com/securifera/e7eed730cbe1ce43d0c29d7cd2d582f4
with open("data/extensions.txt", "r") as f:
    for line in f.readlines():
        extensions.add(line.strip().casefold())
temp = "|".join(map(re.escape, extensions))
file_type_pattern = re.compile(rf"(?:{temp})$")

df["file_suffix"] = df["text"].astype(str).str.contains(url_pattern, na=False)
display(df)

In [ ]:
df["string_length"] = df["text"].astype(str).str.len()

In [ ]:
from collections import defaultdict

key_distances = pd.read_csv("/Users/danylewin/thingies/university/CS Workshop/Finney/data/bigrams.csv",
                            index_col=0).to_numpy()
key_index = {ch: i for i, ch in enumerate("!@#$%^&*()_+1234567890-=qwertyuiop[]{}asdfghjkl;'\\:\"|~zxcvbnm,./<>?)}")}

character_type_map = defaultdict(int)
for c in "abcdefghijklmnopqrstuvwxyz":
    character_type_map[c] = 1
for c in "0123456789":
    character_type_map[c] = 2


def extract_bigrams(word: str) -> list[tuple[str, str]]:
    if type(word) != str or len(word) < 2:
        return []
    word = word.lower().strip()
    return list(zip(word, word[1:]))


def avg_key_distance(bigrams: list[tuple[str, str]]) -> np.float32:
    total_distance = np.float32(0)
    if not bigrams:
        return total_distance
    for bigram in bigrams:
        c1, c2 = bigram
        if c1 not in key_index or c2 not in key_index:
            continue
        total_distance += key_distances[key_index[c1]][key_index[c2]]  # type: ignore
    return total_distance / len(bigrams)


def count_type_switches(bigrams: list[tuple[str, str]]) -> int:
    switch_count = 0
    for bigram in bigrams:
        c1, c2 = bigram
        switch_count += character_type_map[c1] != character_type_map[c2]
    return switch_count


def extract_trigrams(word: str) -> list[tuple[str, str, str]]:
    if type(word) != str or len(word) < 3:
        return []
    word = word.lower().strip()
    return list(zip(word, word[1:], word[2:]))


def has_consecutive_sequence(trigrams: list[tuple[str, str, str]]) -> bool:
    for trigram in trigrams:
        c1, c2, c3 = trigram
        if abs(ord(c1) - ord(c2)) == 1 and ord(c1) - ord(c2) == ord(c2) - ord(c3):
            return True
    return False

In [ ]:
# split the word into bigrams (e.g. "bigram" -> [bi, ig, gr, ra, am])
bigrams = df.text.map(lambda word: extract_bigrams(word))
df["key_distances"] = list(map(avg_key_distance, bigrams))
df["type_switches"] = list(map(count_type_switches, bigrams))

# split the word into trigrams (e.g. "trigram" -> [tri, rig, igr, gra, ram])
trigrams = df.text.map(lambda word: extract_trigrams(word))
df["consecutive_sequence"] = list(map(has_consecutive_sequence, trigrams))

In [ ]:
df.sample(10)

In [ ]:
import numpy as np
def stats(data):
    # Example: identify column types
    bool_cols = data.select_dtypes(include=bool).columns
    obj_cols = data.select_dtypes(include=object).columns.difference(['label', 'text'])
    num_cols = data.select_dtypes(include=[np.number]).columns.difference(['label'])
    # (exclude 'label' itself if it's numeric)

    # Build aggregation dictionary
    agg_dict = {col: 'sum' for col in bool_cols}  # count of True
    agg_dict.update({col: 'sum' for col in obj_cols})  # count of True
    agg_dict.update({col: 'mean' for col in num_cols})  # average for numeric

    # Group by label
    results = data.groupby('label').agg(agg_dict).reset_index()
    return results


print(stats(df))

In [ ]:
df.groupby('label').count()

In [ ]:
def to_csv(data: pd.DataFrame):
    data.to_csv("feature_stats.csv", index=False)

# to_csv(stats(df))
# df.to_csv("feature_stats_raw.csv", index=False)

In [ ]:
print(df.dtypes)

In [ ]:
import matplotlib.pyplot as plt

out_path = "plots"

int_cols = [
    {
        "title": "Longest uppercase sequence",
        "x_axis": "Sequence length",
        "column": "longest_upper"
    },
    {
        "title": "Longest vowel sequence",
        "x_axis": "Sequence length",
        "column": "longest_vowels"
    },
    {
        "title": "Longest consonant sequence",
        "x_axis": "Sequence length",
        "column": "longest_cons"
    },
    {
        "title": "Longest hexadecimal sequence",
        "x_axis": "Sequence length",
        "column": "longest_hexa"
    },
    {
        "title": "Number of sequences of length divisible by 4",
        "x_axis": "# of sequences",
        "column": "word_length_mod_4"
    },
    {
        "title": "English words",
        "x_axis": "# of contained English words",
        "column": "word_count"
    },
    {
        "title": "Programming keywords",
        "x_axis": "# of contained programming keywords",
        "column": "keyword_count"
    },
    {
        "title": "String length",
        "x_axis": "Length",
        "column": "string_length"
    },
    {
        "title": "Character type switches",
        "x_axis": "# of type switches",
        "column": "type_switches"
    },
]

labels = {
    0: "Ordinary string",
    1: "Human password",
    2: "Computer password"
}

for col in int_cols:
    plt.figure(figsize=(10, 5))
    for label in [0, 1, 2]:
        series = df[df.label == label][col["column"]].dropna().value_counts(sort=False).sort_index()
        plt.plot(series.index, series, label=labels[label])
        plt.xlabel(col["x_axis"], fontweight="bold")
    plt.ylabel("Count (log scale)", fontweight="bold")
    plt.legend(loc="upper right")
    plt.title(
        col["title"],
        fontsize=16,
        color="brown",
        fontweight="bold",
        fontstyle="italic",
    )
    plt.savefig(f"plots/{col['column']}.png")
    print(f"Saved result to {col['column']}")


In [ ]:
for col in ["format"]:
    plt.figure(figsize=(10, 5))
    for label in [0, 1, 2]:
        series = df[df.label == label]["format"].dropna().value_counts(sort=False).sort_index()
        plt.plot(["snake_case", "PascalCase", "camelCase", "kebab-case", "ALLCAPS"], series[1:], label=labels[label])
        plt.xlabel("String Format", fontweight="bold")
    plt.ylabel("Count", fontweight="bold")
    plt.legend(loc="upper right")
    plt.title(
        "Special string formats",
        fontsize=16,
        color="brown",
        fontweight="bold",
        fontstyle="italic",
    )
    plt.savefig(f"plots/format.png")
    print(f"Saved result to format")
    
    print(df.groupby(["label", "format"]).count())
